In [13]:
import math
import torch
from torch import nn
from d2l import torch as d2l

In [46]:
#@save
def masked_softmax(X, valid_lens):
    """通过在最后一个轴上掩蔽元素来执行softmax操作"""
    # X:3D张量，valid_lens:1D或2D张量
    if valid_lens is None:
        return nn.functional.softmax(X, dim=-1)
    else:
        shape = X.shape
        print('valid_lens.dim()', valid_lens.dim())
        if valid_lens.dim() == 1:
            valid_lens = torch.repeat_interleave(valid_lens, shape[1])
            print('valid_lens', valid_lens)
        else:
            valid_lens = valid_lens.reshape(-1)
        # 最后一轴上被掩蔽的元素使用一个非常大的负值替换，从而其softmax输出为0
        print('X.reshape(-1, shape[-1])', X.reshape(-1, shape[-1]).shape)
        X = d2l.sequence_mask(X.reshape(-1, shape[-1]), valid_lens,
                              value=-1e6)
        return nn.functional.softmax(X.reshape(shape), dim=-1)

In [47]:
masked_softmax(torch.rand(3, 2, 4), torch.tensor([2,3,2]))

valid_lens.dim() 1
valid_lens tensor([2, 2, 3, 3, 2, 2])
X.reshape(-1, shape[-1]) torch.Size([6, 4])


tensor([[[0.4971, 0.5029, 0.0000, 0.0000],
         [0.4671, 0.5329, 0.0000, 0.0000]],

        [[0.2523, 0.3416, 0.4061, 0.0000],
         [0.4595, 0.3008, 0.2397, 0.0000]],

        [[0.6809, 0.3191, 0.0000, 0.0000],
         [0.4557, 0.5443, 0.0000, 0.0000]]])

In [20]:
#@save
class AdditiveAttention(nn.Module):
    """加性注意力"""
    def __init__(self, key_size, query_size, num_hiddens, dropout, **kwargs):
        super(AdditiveAttention, self).__init__(**kwargs)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=False)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=False)
        self.w_v = nn.Linear(num_hiddens, 1, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, queries, keys, values, valid_lens):
        # 线性变化前：
        # queries:[batch_size,key_size,num_hidden]
        # key的形状：(batch_size，query_size，num_hiddens)
        # 线性变化后：
        # queries:[batch_size,查询的个数,num_hidden]
        # key的形状：(batch_size，“键－值”对的个数，num_hiddens)
        queries, keys = self.W_q(queries), self.W_k(keys)
        # 在维度扩展后，
        # queries的形状：(batch_size，查询的个数，1，num_hidden)
        # key的形状：(batch_size，1，“键－值”对的个数，num_hiddens)
        # 使用广播方式进行求和
        features = queries.unsqueeze(2) + keys.unsqueeze(1)
        print('features',features.shape)
        features = torch.tanh(features)
        # self.w_v仅有一个输出，因此从形状中移除最后那个维度。
        # scores的形状：(batch_size，查询的个数，“键-值”对的个数)
        scores = self.w_v(features).squeeze(-1)
        self.attention_weights = masked_softmax(scores, valid_lens)
        print('attention_weights', self.attention_weights)
        # values的形状：(batch_size，“键－值”对的个数，值的维度)
        return torch.bmm(self.dropout(self.attention_weights), values)

In [21]:
queries, keys = torch.normal(0, 1, (2, 1, 20)), torch.ones((2, 10, 2))
# values的小批量，两个值矩阵是相同的
values = torch.arange(40, dtype=torch.float32).reshape(1, 10, 4).repeat(
    2, 1, 1)
valid_lens = torch.tensor([2, 6])

attention = AdditiveAttention(key_size=2, query_size=20, num_hiddens=8,
                              dropout=0.1)
attention.eval()
attention(queries, keys, values, valid_lens)

features torch.Size([2, 1, 10, 8])
attention_weights tensor([[[0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000]],

        [[0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000,
          0.0000, 0.0000]]], grad_fn=<SoftmaxBackward0>)


tensor([[[ 2.0000,  3.0000,  4.0000,  5.0000]],

        [[10.0000, 11.0000, 12.0000, 13.0000]]], grad_fn=<BmmBackward0>)

## Transformer图片

<img src="https://zh-v2.d2l.ai/_images/transformer.svg" width="500px" style="margin-left:0">

In [54]:
#@save
class DotProductAttention(nn.Module):
    """缩放点积注意力"""
    def __init__(self, dropout, **kwargs):
        super(DotProductAttention, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)

    # queries的形状：(batch_size，查询的个数，d)
    # keys的形状：(batch_size，“键－值”对的个数，d)
    # values的形状：(batch_size，“键－值”对的个数，值的维度)
    # valid_lens的形状:(batch_size，)或者(batch_size，查询的个数)
    def forward(self, queries, keys, values, valid_lens=None):
        print('queries', queries.shape)
        print('keys', keys.shape)
        d = queries.shape[-1]
        # 设置transpose_b=True为了交换keys的最后两个维度
        scores = torch.bmm(queries, keys.transpose(1,2)) / math.sqrt(d)
        print('scores', scores.shape)
        print('scores', scores)
        print('valid_lens', valid_lens.shape)
        self.attention_weights = masked_softmax(scores, valid_lens)
        print('self.attention_weights', self.attention_weights)
        print('self.dropout(self.attention_weights)', self.dropout(self.attention_weights))
        print('torch.bmm(self.dropout(self.attention_weights), values)', torch.bmm(self.dropout(self.attention_weights), values).shape)
        return torch.bmm(self.dropout(self.attention_weights), values)

In [55]:
queries = torch.normal(0, 1, (2, 1, 2))
attention = DotProductAttention(dropout=0.5)
attention.eval()
valid_lens = torch.tensor([2, 6])
values = torch.arange(40, dtype=torch.float32).reshape(1, 10, 4).repeat(2, 1, 1)
attention(queries, keys, values, valid_lens)

queries torch.Size([2, 1, 2])
keys torch.Size([2, 10, 2])
scores torch.Size([2, 1, 10])
scores tensor([[[ 0.1913,  0.1913,  0.1913,  0.1913,  0.1913,  0.1913,  0.1913,
           0.1913,  0.1913,  0.1913]],

        [[-0.0349, -0.0349, -0.0349, -0.0349, -0.0349, -0.0349, -0.0349,
          -0.0349, -0.0349, -0.0349]]])
valid_lens torch.Size([2])
valid_lens.dim() 1
valid_lens tensor([2, 6])
X.reshape(-1, shape[-1]) torch.Size([2, 10])
self.attention_weights tensor([[[0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000]],

        [[0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000,
          0.0000, 0.0000]]])
self.dropout(self.attention_weights) tensor([[[0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000]],

        [[0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000,
          0.0000, 0.0000]]])
torch.bmm(self.dropout(self.attention_weights), values) torch.Size([2, 1, 4])


tensor([[[ 2.0000,  3.0000,  4.0000,  5.0000]],

        [[10.0000, 11.0000, 12.0000, 13.0000]]])